<a href="https://colab.research.google.com/github/aksanaboo/sephora/blob/main/SephoraRecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
#datasets
product = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/product_info.csv')
review_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_0-250.csv')
review_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_250-500.csv')
review_3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_500-750.csv')
review_4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_750-1250.csv')
review_5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_1250-end.csv')

<ipython-input-3-e4867818210a>:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_0-250.csv')
<ipython-input-3-e4867818210a>:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_750-1250.csv')
<ipython-input-3-e4867818210a>:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_1250-end.csv')


In [4]:
df = [review_1,review_2,review_3,review_4,review_5]
#concat all reviews into one
review = pd.concat(df,axis=0)

In [5]:
review_data = review[['product_name','brand_name','review_text','review_title','price_usd']]

In [6]:
review_data = review_data.dropna(subset=['review_text'])
review_data.fillna(' ', inplace=True)

In [7]:
review_data.isnull().sum()

product_name    0
brand_name      0
review_text     0
review_title    0
price_usd       0
dtype: int64

In [10]:
review_data['content'] = review_data['product_name'].astype(str) + ' ' + review_data['review_text'].astype(str) + ' ' + review_data['review_title'].astype(str) + ' ' + review_data['brand_name'].astype(str)

In [11]:
review_data.head()

,product_name,brand_name,review_text,review_title,price_usd,content
0,Gentle Hydra-Gel Face Cleanser,NUDESTIX,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,19.0,Gentle Hydra-Gel Face Cleanser I use this with...
1,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,I bought this lip mask after reading the revie...,Disappointed,24.0,Lip Sleeping Mask Intense Hydration with Vitam...
2,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,My review title says it all! I get so excited ...,New Favorite Routine,24.0,Lip Sleeping Mask Intense Hydration with Vitam...
3,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,I’ve always loved this formula for a long time...,Can't go wrong with any of them,24.0,Lip Sleeping Mask Intense Hydration with Vitam...
4,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,"If you have dry cracked lips, this is a must h...",A must have !!!,24.0,Lip Sleeping Mask Intense Hydration with Vitam...


In [12]:
review_grouped = review_data.groupby('product_name')['content'].apply(lambda x: ', '.join(x)).reset_index()

In [17]:
review_grouped.head()

,product_name,content
0,"""B"" Oil","""B"" Oil It’s a little bit golden-green-brown i..."
1,"""Buffet"" + Copper Peptides 1%","""Buffet"" + Copper Peptides 1% Has helped my ey..."
2,(Glow)Setting 100% Mineral Powder SPF 35,(Glow)Setting 100% Mineral Powder SPF 35 I lov...
3,(Re) Setting Refreshing Mist SPF 40,(Re) Setting Refreshing Mist SPF 40 Pump stopp...
4,(Re)setting 100% Mineral Powder Sunscreen SPF ...,(Re)setting 100% Mineral Powder Sunscreen SPF ...


In [28]:
review_grouped.product_name[20]

'100% Mineral SPF Starter Kit'

In [14]:
#preprocess textual data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    # Join tokens to form preprocessed text
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [18]:
review_grouped['content'] = review_grouped['content'].apply(preprocess_text)

In [20]:
# Create bag of words
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(review_grouped['content'])

In [23]:
# Convert bag of words to TF-IDF
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(bow)

In [24]:
# Apply LSA or LSI
lsa = TruncatedSVD(n_components=100, algorithm='arpack')
lsa.fit(tfidf)

TruncatedSVD(algorithm='arpack', n_components=100)

## Finding Similar Products based on Product name

In [29]:
# Get the user input
user_product = input("Enter a Product name: ")

Enter a Product name: 100% Mineral SPF Starter Kit


In [32]:
try:
    product_index = review_grouped[review_grouped['product_name'] == user_product].index[0]
    similarity_scores = cosine_similarity(tfidf[product_index], tfidf)
    similar_products = list(enumerate(similarity_scores[0]))
    sorted_similar_products = sorted(similar_products, key=lambda x: x[1], reverse=True)[1:11]

    # Print the top 10 similar products
    for i, score in sorted_similar_products:
        print("{}: {}".format(i, review_grouped.loc[i, 'product_name']))

except IndexError:
    print("Product not found in the dataset.")
except KeyError as e:
    print(f"An error occurred while accessing the DataFrame: {e}")

21: 100% Mineral Sunscreen Starter Kit
1485: PLAY 100% Mineral Lotion SPF 30 with Green Algae
1306: Mini Mineral Sheerscreen Sunscreen SPF 30 PA+++
1207: Mineral Sheerscreen Sunscreen SPF 30 PA+++
714: Every. Single. Face. Watery Lotion SPF 50
2324: Zincscreen 100% Mineral Sunscreen Lotion SPF 40 PA+++
1552: Play! Mineral SPF Stick
4: (Re)setting 100% Mineral Powder Sunscreen SPF 35 PA+++
1305: Mini Mineral Mattescreen Sunscreen SPF 40 PA+++
1775: SPF Bestsellers Kit


## Finding products based on User description

In [35]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(review_grouped['content'])

In [36]:
# Calculate cosine similarity between products
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [37]:
def recommend_products(user_description, num_recommendations=5):
    # Preprocess user's description
    processed_user_description = preprocess_text(user_description)

    # Convert user's description to TF-IDF vector
    user_tfidf = tfidf_vectorizer.transform([processed_user_description])

    # Calculate cosine similarity between user's TF-IDF vector and product TF-IDF vectors
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix)

    # Get indices of top N similar products
    top_indices = similarity_scores.argsort()[0][::-1][:num_recommendations]

    # Recommend products
    recommended_products = review_grouped.loc[top_indices, 'product_name'].tolist()

    return recommended_products

In [44]:
user_description = input('Describe the type of product you are looking for: ')

Describe the type of product you are looking for: a great vitamin C moisturizer


In [43]:
recommended_products = recommend_products(user_description, num_recommendations=5)
print("Recommended Products:", recommended_products)

Recommended Products: ['Vitamin C Glow Moisturizer', 'Vitamin B, C, and E Moisturizer', 'Peptide Moisturizer', 'Ultra Repair Face Moisturizer', 'Oil Free Moisturizer']
